**1.0 Insert Packages**

In [1]:
!pip install geopy
!pip install geocoder
!pip install geopy
!pip install folium

     |████████████████████████████████| 102kB 6.6MB/s ta 0:00:011
     |████████████████████████████████| 102kB 7.1MB/s ta 0:00:011


**2.0 Import Libraries**

In [2]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.") 

Libraries imported.


**3.0 Scrape Data from Wikipedia**

In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Auckland").text

In [4]:
# parse data from the html into a beautifulsoup objec
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
# create a new DataFrame from the list
Auck_df = pd.DataFrame({"Neighborhood": neighborhoodList})

Auck_df.head(15)

,Neighborhood
0,List of suburbs of Auckland
1,Airport Oaks
2,"Albany, New Zealand"
3,"Alfriston, New Zealand"
4,Algies Bay
5,"Arch Hill, New Zealand"
6,"Ardmore, New Zealand"
7,Army Bay
8,Auckland CBD
9,Auckland waterfront


In [8]:
# print the number of rows of the dataframe
Auck_df.shape

(200, 1)

In [11]:
Auck_df = Auck_df.drop(Auck_df.index[0])
Auck_df.head(10)

,Neighborhood
1,Airport Oaks
2,"Albany, New Zealand"
3,"Alfriston, New Zealand"
4,Algies Bay
5,"Arch Hill, New Zealand"
6,"Ardmore, New Zealand"
7,Army Bay
8,Auckland CBD
9,Auckland waterfront
10,"Avondale, Auckland"


**4.0 Obtain coordinates**

In [12]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Auckland, New Zealand'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [13]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in Auck_df["Neighborhood"].tolist() ]

In [14]:
coords

[[-36.79359048580961, 174.61801235305123],
 [-36.72915999999998, 174.70893000000012],
 [-37.010493372999974, 174.9408516090001],
 [-36.84839904099994, 174.76438785300002],
 [-36.86301999999995, 174.74858000000006],
 [-36.847859693114465, 174.73708327694862],
 [-36.59818999999993, 174.81019000000003],
 [-36.84839904099994, 174.76438785300002],
 [-36.84839904099994, 174.76438785300002],
 [-36.89044821899995, 174.6870174820001],
 [-36.88819999999998, 174.7401900000001],
 [-36.80819999999994, 174.77019000000007],
 [-36.77452705999997, 174.7090190230001],
 [-36.69486999999998, 174.75065000000006],
 [-36.80523647199993, 174.788186794],
 [-36.797416829999975, 174.70436862300005],
 [-36.80892881099993, 174.72194536900008],
 [-36.92819999999995, 174.70019000000002],
 [-36.90945304999997, 174.9170680730001],
 [-36.98755915999993, 174.9893839770001],
 [-36.71926010899995, 174.73666409700002],
 [-36.858199999999954, 174.90019000000007],
 [-36.74957706799995, 174.75627076500007],
 [-36.760500193999

In [15]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [16]:
# merge the coordinates into the original dataframe
Auck_df['Latitude'] = df_coords['Latitude']
Auck_df['Longitude'] = df_coords['Longitude']

In [17]:
print(Auck_df.shape)
Auck_df

(199, 3)


,Neighborhood,Latitude,Longitude
1,Airport Oaks,-36.729160,174.708930
2,"Albany, New Zealand",-37.010493,174.940852
3,"Alfriston, New Zealand",-36.848399,174.764388
4,Algies Bay,-36.863020,174.748580
5,"Arch Hill, New Zealand",-36.847860,174.737083
6,"Ardmore, New Zealand",-36.598190,174.810190
7,Army Bay,-36.848399,174.764388
8,Auckland CBD,-36.848399,174.764388
9,Auckland waterfront,-36.890448,174.687017
10,"Avondale, Auckland",-36.888200,174.740190


In [18]:
# Dropping the Neighborhood with missing value
Auck_df = Auck_df.drop(Auck_df.index[198])
Auck_df.tail(10)

,Neighborhood,Latitude,Longitude
189,"Three Kings, New Zealand",-36.940495,174.647251
190,Titirangi,-36.699000,174.736330
191,Torbay Heights,-36.694870,174.750650
192,"Torbay, New Zealand",-37.001953,174.900534
193,Totara Heights,-36.762727,174.728084
194,Totara Vale,-36.757412,174.716526
195,Unsworth Heights,-36.818200,174.800190
196,"Vauxhall, Auckland",-36.845280,174.758450
197,Viaduct Harbour,-36.868200,174.870190
198,Wai o Taiki Bay,-36.706590,174.749193


In [19]:
# save the DataFrame as CSV file
Auck_df.to_csv("Auck_df.csv", index=False)

**5.0 Create a map of Auckland with Neighborhoods**

In [20]:
# get the coordinates of Auckland
address = 'Auckland, New Zealand'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Auckland, New Zealand {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Auckland, New Zealand -36.852095, 174.7631803.


In [21]:
# create map of Toronto using latitude and longitude values
map_Auck = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(Auck_df['Latitude'], Auck_df['Longitude'], Auck_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Auck)  
    
map_Auck

In [22]:
map_Auck.save('map_Auck.html')

**6.0 Use Foursquare API to explore Neighborhoods**

In [45]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20201307' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


In [24]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(Auck_df['Latitude'], Auck_df['Longitude'], Auck_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [25]:
# Convert venues list into new dataframe
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()


(7806, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Airport Oaks,-36.72916,174.70893,K-Mart,-36.728644,174.709722,Department Store
1,Airport Oaks,-36.72916,174.70893,QBE Stadium,-36.726937,174.702059,Stadium
2,Airport Oaks,-36.72916,174.70893,Event Cinemas,-36.728335,174.708643,Movie Theater
3,Airport Oaks,-36.72916,174.70893,Matsu Sushi,-36.728465,174.708699,Sushi Restaurant
4,Airport Oaks,-36.72916,174.70893,The Merchant Bar & Kitchen,-36.727810,174.709088,Bar


In [26]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Airport Oaks,61,61,61,61,61,61
"Albany, New Zealand",1,1,1,1,1,1
"Alfriston, New Zealand",100,100,100,100,100,100
Algies Bay,100,100,100,100,100,100
"Arch Hill, New Zealand",100,100,100,100,100,100
"Ardmore, New Zealand",6,6,6,6,6,6
Army Bay,100,100,100,100,100,100
Auckland CBD,100,100,100,100,100,100
Auckland waterfront,38,38,38,38,38,38


In [27]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 250 uniques categories.


In [28]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Department Store', 'Stadium', 'Movie Theater', 'Sushi Restaurant',
       'Bar', 'Shopping Mall', 'Bubble Tea Shop', 'Grocery Store',
       'Multiplex', 'Burger Joint', 'Supermarket', 'Fast Food Restaurant',
       'Portuguese Restaurant', 'Café', 'Furniture / Home Store',
       'Electronics Store', 'Coffee Shop', 'Juice Bar',
       'Seafood Restaurant', 'Sporting Goods Shop', 'Restaurant',
       'American Restaurant', 'Garden Center', 'Japanese Restaurant',
       'Park', 'Arts & Crafts Store', 'Thai Restaurant', 'Optical Shop',
       'Noodle House', 'Mexican Restaurant', 'Bus Station', 'BBQ Joint',
       'Asian Restaurant', 'Food & Drink Shop', 'Playground', 'Pet Store',
       'Korean Restaurant', 'Other Great Outdoors', 'Nature Preserve',
       'Chinese Restaurant', 'Pharmacy', 'Gym', 'Bakery', 'Deli / Bodega',
       'Theater', 'Bistro', 'Food Court', 'Monument / Landmark',
       'Art Gallery', 'Tapas Restaurant'], dtype=object)

In [29]:
# check if the results contain "Restaurant"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

**7.0 Analyze each Neighborhood**

In [30]:
# one hot encoding
Auck_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Auck_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Auck_onehot.columns[-1]] + list(Auck_onehot.columns[:-1])
Auck_onehot = Auck_onehot[fixed_columns]

print(Auck_onehot.shape)
Auck_onehot.head()

(7806, 251)


,Neighborhoods,African Restaurant,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Stadium,Bay,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Chinese Restaurant,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Heliport,Hill,History Museum,Hobby Shop,Hockey Field,Home Service,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,Night Market,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rest Area,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo
0,Airport Oaks,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [31]:
Auck_grouped = Auck_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(Auck_grouped.shape)
Auck_grouped

(194, 251)


,Neighborhoods,African Restaurant,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Stadium,Bay,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Chinese Restaurant,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Heliport,Hill,History Museum,Hobby Shop,Hockey Field,Home Service,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,Night Market,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rest Area,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo
0,Airport Oaks,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.016393,0.016393,0.000000,0.000000,0.00,0.00,0.016393,0.000000,0.00,0.000000,0.049180,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.016393,0.000000,0.000000,0.016393,0.016393,0.00,0.000000,0.000000,0.131148,0.00,0.000000,0.000000,0.000000,0.016393,0.00,0.000000,0

In [32]:
len(Auck_grouped[Auck_grouped["Restaurant"] > 0])

87

In [33]:
Auck_rest = Auck_grouped[["Neighborhoods","Restaurant"]]

In [34]:
Auck_rest.head()

,Neighborhoods,Restaurant
0,Airport Oaks,0.016393
1,"Albany, New Zealand",0.000000
2,"Alfriston, New Zealand",0.040000
3,Algies Bay,0.060000
4,"Arch Hill, New Zealand",0.060000


**8.0 Cluster Neighborhoods**

In [35]:
# set number of clusters
kclusters = 3

Auck_clustering = Auck_rest.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Auck_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 2, 2, 0, 2, 2, 0, 0], dtype=int32)

In [36]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Auck_merged = Auck_rest.copy()

# add clustering labels
Auck_merged["Cluster Labels"] = kmeans.labels_

In [37]:
Auck_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
Auck_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,Airport Oaks,0.016393,0
1,"Albany, New Zealand",0.000000,0
2,"Alfriston, New Zealand",0.040000,2
3,Algies Bay,0.060000,2
4,"Arch Hill, New Zealand",0.060000,2


In [38]:
# merge Auckland_grouped with Auckland_data to add latitude/longitude for each neighborhood
Auck_merged = Auck_merged.join(Auck_df.set_index("Neighborhood"), on="Neighborhood")

print(Auck_merged.shape)
Auck_merged.head() # check the last columns!

(194, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Airport Oaks,0.016393,0,-36.729160,174.708930
1,"Albany, New Zealand",0.000000,0,-37.010493,174.940852
2,"Alfriston, New Zealand",0.040000,2,-36.848399,174.764388
3,Algies Bay,0.060000,2,-36.863020,174.748580
4,"Arch Hill, New Zealand",0.060000,2,-36.847860,174.737083


In [39]:
# sort the results by Cluster Labels
print(Auck_merged.shape)
Auck_merged.sort_values(["Cluster Labels"], inplace=True)
Auck_merged

(194, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Airport Oaks,0.016393,0,-36.729160,174.708930
145,"Ponsonby, New Zealand",0.000000,0,-36.865026,174.601326
148,Remuera,0.000000,0,-36.884002,174.681984
91,Manukau,0.013333,0,-36.890447,174.774965
90,Mairangi Bay,0.000000,0,-36.980840,174.781594
89,"Lynfield, New Zealand",0.000000,0,-36.888200,174.610190
88,"Long Bay, New Zealand",0.000000,0,-36.930359,174.719858
149,"Rosebank, Auckland",0.000000,0,-36.746500,174.715221
86,Laingholm,0.000000,0,-36.858200,174.620190
85,Konini,0.000000,0,-36.963937,174.630086


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Auck_merged['Latitude'], Auck_merged['Longitude'], Auck_merged['Neighborhood'], Auck_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [41]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

**9.0 Examine Clusters**

In [42]:
Auck_merged.loc[Auck_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Airport Oaks,0.016393,0,-36.729160,174.708930
145,"Ponsonby, New Zealand",0.000000,0,-36.865026,174.601326
148,Remuera,0.000000,0,-36.884002,174.681984
91,Manukau,0.013333,0,-36.890447,174.774965
90,Mairangi Bay,0.000000,0,-36.980840,174.781594
89,"Lynfield, New Zealand",0.000000,0,-36.888200,174.610190
88,"Long Bay, New Zealand",0.000000,0,-36.930359,174.719858
149,"Rosebank, Auckland",0.000000,0,-36.746500,174.715221
86,Laingholm,0.000000,0,-36.858200,174.620190
85,Konini,0.000000,0,-36.963937,174.630086


In [43]:
Auck_merged.loc[Auck_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
159,"Sandspit, New Zealand",0.071429,1,-36.756174,174.686915
181,"Three Kings, New Zealand",0.100000,1,-36.940495,174.647251
183,Torbay Heights,0.076923,1,-36.694870,174.750650
182,Titirangi,0.064516,1,-36.699000,174.736330
155,Rānui,0.076923,1,-36.694870,174.750650
156,Saint Heliers,0.078947,1,-36.838200,174.740190
144,Point England,0.070000,1,-36.848140,174.743700
136,Papakura North,0.071429,1,-36.976920,174.855379
96,"Massey, New Zealand",0.083333,1,-36.618190,174.770190
193,Ōtāhuhu,0.071429,1,-36.962170,174.874850


In [44]:
Auck_merged.loc[Auck_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
17,Blockhouse Bay,0.030303,2,-36.909453,174.917068
54,Goodwood Heights,0.040000,2,-36.860880,174.769480
50,"Glendene, New Zealand",0.040000,2,-36.860361,174.872659
191,Ōpaheke,0.022727,2,-36.850000,174.816670
190,Wai o Taiki Bay,0.025000,2,-36.706590,174.749193
172,Takanini,0.021978,2,-36.787932,174.765549
2,"Alfriston, New Zealand",0.040000,2,-36.848399,174.764388
174,Tamaki City,0.026316,2,-36.847790,174.829290
37,"Eastern Beach, New Zealand",0.040000,2,-36.864135,174.757433
188,"Vauxhall, Auckland",0.060000,2,-36.845280,174.758450


**Conclusion**

Most of the restaurants are concentrated in cluster 1 and  cluster 2. On the other hand, cluster 0 has  low number to totally no restaurant in the neighborhoods. This represents a great opportunity and high potential areas to open new restaurant as there is very little to no competition from existing restaurants. Meanwhile, restaurants in cluster 1 and cluster 2 are likely suffering from intense competition due to oversupply and high concentration of restaurants. 

Therefore, this project recommends investors to capitalize on these findings to open new restaurant in neighborhoods in cluster 0 with little to no competition. Lastly, investors are advised to avoid neighborhoods in cluster 1 and cluster 2, which already have high concentration of restaurants and suffering from intense competition.